# Python 描述符

每一个对象均有 `__dict__` 属性，它的作用是：字典类型，存放对象的属性。

对象属性的访问顺序：

1. 实例属性
2. 类属性
3. 父属性
4. `__getattr__` 方法

In [49]:
class Test:
    cls_val = 1
    def __init__(self):
        self.ins_val = 10

In [52]:
Test.__dict__

mappingproxy({'__module__': '__main__',
              'cls_val': 1,
              '__init__': <function __main__.Test.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'Test' objects>,
              '__weakref__': <attribute '__weakref__' of 'Test' objects>,
              '__doc__': None})

In [51]:
t = Test()
t.__dict__

{'ins_val': 10}

In [54]:
type(t)

__main__.Test

更改实例 `t` 的属性 `cls_val`, 只是新增了该属性，并不影响类 `Test` 的属性 `cls_val`：

In [55]:
t.cls_val = 20
t.__dict__

{'ins_val': 10, 'cls_val': 20}

In [57]:
Test.__dict__

mappingproxy({'__module__': '__main__',
              'cls_val': 1,
              '__init__': <function __main__.Test.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'Test' objects>,
              '__weakref__': <attribute '__weakref__' of 'Test' objects>,
              '__doc__': None})

更改类 `Test` 的属性 `cls_val`, 由于事先增加了实例 `t` 的 `cls_val` 属性，因此不会改变实例 `t` 的属性 `cls_val` (井水不犯河水)：

In [58]:
Test.cls_val = 30

In [59]:
t.__dict__

{'ins_val': 10, 'cls_val': 20}

In [60]:
Test.__dict__

mappingproxy({'__module__': '__main__',
              'cls_val': 30,
              '__init__': <function __main__.Test.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'Test' objects>,
              '__weakref__': <attribute '__weakref__' of 'Test' objects>,
              '__doc__': None})

从上面的分析可以看出：实例 `t` 的属性并不包含 `cls_val`, `cls_val` 属于类 `Test`。

## 魔法方法

- `__get__(self, instance, owner)`
- `__set__(self, instance, value)`
- `__del__(self, instance)`

In [80]:
class Desc:
    def __get__(self, instance, owner):
        '''
        返回属性对象
        '''
        print('__get__...')
        print('self: \t\t', self)
        print('instance:\t', instance)
        print('owner: \t', owner)
        print('='*50,'\n')
        
    def __set__(self, instance, value):
        print('__set__...')
        print('self: \t\t', self)
        print('instance:\t', instance)
        print('value: \t', value)
        print('='*50,'\n')

In [81]:
class TestDesc:
    x = Desc()

In [82]:
t = TestDesc()

t.x

__get__...
self: 		 <__main__.Desc object at 0x0000020F83CB7BA8>
instance:	 <__main__.TestDesc object at 0x0000020F83CB7240>
owner: 	 <class '__main__.TestDesc'>



可以看出实例化之后，调用对象 `t` 的属性 `x`，会自动调用类 `Desc` 的 `__get__` 方法，且：

- `self`：`Desc` 的实例对象，其实就是 `TestDesc` 的属性 `x`
- `instance`：`TestDesc` 的实例对象，其实就是 `t`
- `owner`：即谁拥有这些东西，当然是 `testDesc` 这个类，它是最高统治者，其他的一些都是包含在它内部或者由它生成的

# 装饰器

具有**自动重绑定**操作。

## 函数装饰器

函数装饰器是一种关于函数的**运行时声明**，函数的定义需要遵守此声明。装饰器在紧挨着定义一个函数或方法的 `def` 语句之前的一行编写，并且由 `@` 符号以及紧随其后的对于**元函数**的一个引用组成——这是管理另一个函数的一个函数（或其他的可调用对象）。

In [84]:
def deco(func):  # 修饰器deco
    def wrappedFunc():  # 内嵌函数wrappedFunc(所有对于传入函数的修饰逻辑都将在此内嵌函数中实现。)
        return 'Hello World_'+func()
    return wrappedFunc

# 在程序中若有函数需要修饰器修饰，只需在函数定义前使用“`@+修饰器名`”即可使用此修饰器。


@deco
def f():    # 调用修饰器
    return 'I am f'


def g():     # 没有调用修饰器
    return 'I am g'


print(f())
print(g())

Hello World_I am f
I am g


In [85]:
def deco(f):
    def g():
        return [f()[i] for i in range(5)]
    return g


@deco
def h():
    return [1, 2, 3, 4, 56, 7, '75s']


print(h())

[1, 2, 3, 4, 56]


In [87]:
import time


# 定义一个计时器，传入一个，并返回另一个附加了计时功能的方法
def timeit(func):

    # 定义一个内嵌的包装函数，给传入的函数加上计时功能的包装
    def wrapper():
        start = time.clock()
        func()
        end = time.clock()
        print('used:', end - start)

    # 将包装后的函数返回
    return wrapper


@timeit
def foo():
    print('in foo()')


foo()

in foo()
used: 5.412343540456277e-05
